In [1]:
from bs4 import BeautifulSoup 
import requests 
import pandas as pd
import regex as re
from requests import get 
import json

In [2]:
def fetch_otodom_pages(num_pages):
    # Adres URL, do którego chcesz wysłać żądanie              TU PODMIEN LINK
    base_url = 'https://www.otodom.pl/pl/wyniki/sprzedaz/mieszkanie/kujawsko--pomorskie/bydgoszcz/bydgoszcz/bydgoszcz?viewType=listing'

    # Nagłówek User-Agent
    headers = {
        'User-Agent': 'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Mobile Safari/537.36'
    }

    # Tworzenie pustego DataFrame
    data = pd.DataFrame(columns=['Tytuł', 'Cena', 'Powierzchnia', 'Link', 'Stan wykończenia', 'Liczba pokoi', 'Piętro', 'Ogrzewanie', 'Ogłoszeniodawca', 'Rok budowy', 'Winda', 'Miasto','LAT','LON'])

    # Pobieranie określonej liczby stron
    for page in range(1, num_pages + 1):
        url = f'{base_url}&page={page}'
        response = requests.get(url, headers=headers)
        print(page)

        # Pamiętaj, aby sprawdzić kod statusu odpowiedzi
        if response.status_code == 200:
            web_content = response.text
            bs = BeautifulSoup(web_content, 'html.parser')
            
            # Tutaj możesz przetwarzać zawartość strony i dodawać do DataFrame
            # ...
            data= load_data(bs, data)

        else:
            print(f'Błąd {response.status_code} na stronie {page}: Żądanie nie powiodło się')

    # Zwróć DataFrame z wynikami
    return data
    

In [3]:
def load_data(bs, data):
    for house in bs.find_all('li', class_='css-o9b79t e1dfeild0'):  
        title = house.find('div', class_="css-gg4vpm e1n06ry51")#.get_text()
        price = house.find('div', class_='e1jyrtvq0 css-1tjkj49 ei6hyam0')#.get_text()
        link = house.find('a')
        cena = "Brak dostępnej ceny"
        if price is not None:
            #print(price.text)
            price_pattern = r'(\d*\s\d+\s?\d+)\s+zł'
            price_match = re.search(price_pattern, price.text)
            if price_match:
                cena = price_match.group(1).replace(' ', '')  # Usuń spacje, jeśli istnieją
                #print("Wartość mieszkania:", cena)

            # Wyrażenie regularne do wyciągnięcia powierzchni mieszkania
            area_pattern = r'(\d+(?:\.\d+)?)\s+m²'
            area_match = re.search(area_pattern, price.text)

            if area_match:
                area = area_match.group(1)
                #print("Powierzchnia mieszkania:", area)
                powierchnia =area
            else:
                cena = "Brak dostępnej ceny"
                area = "Brak dostępnej powierzchni"
            if link is not None and 'href' in link.attrs:
                link = 'https://otodom.pl' + link['href']
                result = get_web_content(link)
                if result is not None:
                    bs2, statusWykonczenia, Lpokoi, pietro, ogrzewanie, typOgloszeniodawcy, rokBudowy, winda, materialBudowy, miasto, latitude, longitude = result
                    if bs2 is not None:  # Sprawdzenie, czy get_web_content nie zwróciło None
                        data = data.append({'Tytuł': title.text, 'Cena': cena, 'Powierzchnia': area, 'Link': link,
                                    'Stan wykończenia': statusWykonczenia, 'Liczba pokoi': Lpokoi, 'Piętro': pietro, 'Ogrzewanie': ogrzewanie,
                                         'Ogłoszeniodawca': typOgloszeniodawcy,'Rok budowy': rokBudowy, 'Winda': winda, 
                                            'Materiał budowy': materialBudowy,'Miasto':miasto,'LAT': latitude ,'LON' : longitude  }, ignore_index=True)

                    else:
                        print(f'Błąd pobierania danych z linku')
                else:
                    print("Brak danych dla linku:", link)      
                
            else:
                print('Brak dostępnego linku dla tego domu')
    return data




In [4]:
def get_web_content(url):
    # Nagłówek User-Agent
    headers = {
        'User-Agent': 'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Mobile Safari/537.36'
    }
    # Wysłanie żądania HTTP
    response = requests.get(url, headers=headers)

    # Pamiętaj, aby sprawdzić kod statusu odpowiedzi, żeby upewnić się, że żądanie było udane.
    if response.status_code == 200:
        # Tutaj możesz przetwarzać zawartość strony
        web_content = response.text
        # Assuming 'web_content' contains the HTML content
        bs2 = BeautifulSoup(web_content, 'html.parser')

        # Extract room information
        room_element = bs2.find('div', class_="css-kkaknb enb64yk0", attrs={"aria-label": "Liczba pokoi"})
        room = room_element.get_text() if room_element else "N/A"

        # Extract status information
        status_element = bs2.find('div', class_="css-kkaknb enb64yk0", attrs={"aria-label": "Stan wykończenia"})
        status = status_element.get_text() if status_element else "N/A"

        # Extract floor information
        floor_element = bs2.find('div', class_="css-kkaknb enb64yk0", attrs={"aria-label": "Piętro"})
        floor = floor_element.get_text() if floor_element else "N/A"

        # Extract heating information
        heating_element = bs2.find('div', class_="css-kkaknb enb64yk0", attrs={"aria-label": "Ogrzewanie"})
        heating = heating_element.get_text() if heating_element else "N/A"

        # Extract advertiser information
        advertiser_element = bs2.find('div', class_="css-1k2qr23 enb64yk0", attrs={"aria-label": "Typ ogłoszeniodawcy"})
        advertiser = advertiser_element.get_text() if advertiser_element else "N/A"

        # Extract year of construction information
        yOfConstruction_element = bs2.find('div', class_="css-1k2qr23 enb64yk0", attrs={"aria-label": "Rok budowy"})
        yOfConstruction = yOfConstruction_element.get_text() if yOfConstruction_element else "N/A"

        # Extract elevator information
        elevator_element = bs2.find('div', class_="css-1k2qr23 enb64yk0", attrs={"aria-label": "Winda"})
        elevator = elevator_element.get_text() if elevator_element else "N/A"

        # Extract building material information
        buildinMaterial_element = bs2.find('div', class_="css-1k2qr23 enb64yk0", attrs={"aria-label": "Materiał budynku"})
        buildinMaterial = buildinMaterial_element.get_text() if buildinMaterial_element else "N/A"

        Lpokoi =room[12:]
        statusWykonczenia = status[16:]
        pietro = floor[6:]
        ogrzewanie =heating[10:]
        typOgloszeniodawcy=advertiser[19:]
        rokBudowy = yOfConstruction[10:]
        winda = elevator[5:]
        materialBudowy= buildinMaterial[16:]
        ####################Sztywno Miasto########################################################XX
        miasto = 'Bydgoszcz'
        html_string = bs2.prettify()
        # Wyrażenie regularne do znalezienia danych "latitude" i "longitude"
        pattern = r'"coordinates":{"latitude":(\d+\.\d+),"longitude":(\d+\.\d+)'

        # Wyszukaj dopasowanie w tekście HTML
        match = re.search(pattern, html_string)

        if match:
            latitude = match.group(1)
            longitude = match.group(2)
        else:
            print("Nie znaleziono danych 'latitude' i 'longitude'.")
            latitude = 0
            longitude = 0
        try:
            response = requests.get(url, headers=headers)
            if response.status_code == 200:
                # Przetwarzanie zawartości strony
                if bs2 is None:
                    bs2 = "Brak danych"  # Przypisz wartość neutralną w przypadku braku wartości
                if statusWykonczenia is None:
                    statusWykonczenia = "Brak danych"
                if Lpokoi is None:
                    Lpokoi = 0
                if pietro is None:
                    pietro = 0
                if ogrzewanie is None:
                    ogrzewanie = "Brak danych"
                if typOgloszeniodawcy is None:
                    typOgloszeniodawcy = "Brak danych"
                if rokBudowy is None:
                    rokBudowy = 0
                if winda is None:
                    winda = "Brak danych"
                if materialBudowy is None:
                    materialBudowy = "Brak danych"
                if miasto is None:
                    miasto = "Brak danych"
                if latitude is None:
                    latitude = 0
                if longitude is None:
                    longitude = 0
                return bs2, statusWykonczenia, Lpokoi, pietro, ogrzewanie, typOgloszeniodawcy, rokBudowy, winda, materialBudowy,miasto,latitude,longitude
            else:
                print(f'Błąd {response.status_code}: Żądanie nie powiodło się na stronie {url}')
                return None, None, None, None, None, None, None, None, None , None , None , None
        except Exception as e:
            print(f'Wystąpił błąd: {e}')
            return None, None, None, None, None, None, None, None, None
    else:
        print(f'Błąd {response.status_code}: Żądanie nie powiodło się')
        return None

In [ ]:
df = fetch_otodom_pages(103) #Wpisz liczbe stron z linku który zamiesciles 

In [ ]:
# Funkcja do obliczania ceny za metr kwadratowy
def calculate_price_per_m2(row):
    try:
        cena = float(row['Cena'].replace('\xa0', '').replace(',', '') )
    except (ValueError, TypeError):
        cena = 1.0  # Przypisz wartość 1, jeśli nie można przekonwertować na float

    try:
        powierzchnia = float(row['Powierzchnia'])
    except (ValueError, TypeError):
        powierzchnia = 1.0  # Przypisz wartość 1, jeśli nie można przekonwertować na float

    x = cena / powierzchnia 
    if x <=1:
        return "None"
    else: 
        return x

# Obliczanie ceny za metr kwadratowy i dodanie jej do DataFrame
df['cena za m2'] = df.apply(calculate_price_per_m2, axis=1)

In [ ]:
df.to_csv('OTD_Lodz')